# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sqlalchemy import create_engine
import pickle

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=71)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
from sklearn.metrics import classification_report

Y_pred = pipeline.predict(X_test)

for col in range(36):
    print(Y_test.columns[col])
    print(classification_report(Y_test.iloc[:,col], Y_pred[:,col]))

related
             precision    recall  f1-score   support

          0       0.65      0.38      0.48      1254
          1       0.82      0.93      0.87      3949
          2       0.54      0.17      0.26        41

avg / total       0.78      0.80      0.78      5244

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4344
          1       0.83      0.37      0.51       900

avg / total       0.87      0.88      0.86      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3117
          1       0.75      0.52      0.62      2127

avg / total       0.74      0.74      0.73      5244

medical_help
             precision    recall  f1-sco

Metrics in above is so messy, so I rewrite dataframe form.

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics = []
for i in range(len(Y_test.columns)):
    accuracy = accuracy_score(Y_test.iloc[:, i], Y_pred[:, i])
    precision = precision_score(Y_test.iloc[:, i], Y_pred[:, i], average='micro')
    recall = recall_score(Y_test.iloc[:, i], Y_pred[:, i], average='micro')
    f1 = f1_score(Y_test.iloc[:, i], Y_pred[:, i], average='micro')
    metrics.append([accuracy, precision, recall, f1])

metrics_df = pd.DataFrame(data=np.array(metrics), index=list(Y_test.columns.values), columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
print(metrics_df)

                        Accuracy  Precision    Recall        F1
related                 0.795767   0.795767  0.795767  0.795767
request                 0.878909   0.878909  0.878909  0.878909
offer                   0.995423   0.995423  0.995423  0.995423
aid_related             0.736270   0.736270  0.736270  0.736270
medical_help            0.926583   0.926583  0.926583  0.926583
medical_products        0.955950   0.955950  0.955950  0.955950
search_and_rescue       0.971014   0.971014  0.971014  0.971014
security                0.981884   0.981884  0.981884  0.981884
military                0.966247   0.966247  0.966247  0.966247
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.948894   0.948894  0.948894  0.948894
food                    0.927727   0.927727  0.927727  0.927727
shelter                 0.932685   0.932685  0.932685  0.932685
clothing                0.985698   0.985698  0.985698  0.985698
money                   0.977498   0.977

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'clf__estimator__min_samples_split': [2, 7, 12],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_features': ['auto', 'sqrt', 'log2']
}

cv = GridSearchCV(pipeline, param_grid=parameters)
new_model = cv.fit(X_train, Y_train)
new_model.cv_results_

{'mean_fit_time': array([ 45.74738661,  36.79068319,  34.69786445,  45.52005514,
         37.33417106,  34.70918218,  47.96227535,  35.69904184,
         33.03273924,  46.11771894,  37.45799557,  34.89565802,
         46.32149847,  37.18010759,  35.07809718,  47.76720603,
         36.51225019,  33.34122086]),
 'std_fit_time': array([ 0.51613577,  0.24957224,  0.06135454,  0.54333998,  0.14825049,
         0.48670849,  0.68838663,  0.3393592 ,  0.37363078,  0.53196327,
         0.22495333,  0.29401485,  0.45488932,  0.01172407,  0.43650441,
         0.33191347,  0.31350558,  0.19757313]),
 'mean_score_time': array([ 5.39373724,  5.34476217,  5.34422207,  5.28675636,  5.30658205,
         5.36931578,  6.92245905,  6.8482101 ,  6.84728257,  4.97850752,
         4.90660572,  4.95205013,  4.94908746,  4.91755652,  4.96574418,
         6.74805164,  6.71630836,  6.72045104]),
 'std_score_time': array([ 0.12225935,  0.08291886,  0.04403743,  0.06238323,  0.10832272,
         0.05012906,  0.037

In [9]:
# Find the best parameters
new_model.best_params_

{'clf__estimator__criterion': 'entropy',
 'clf__estimator__max_features': 'sqrt',
 'clf__estimator__min_samples_split': 2}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
new_Y_pred = new_model.predict(X_test)

new_metrics = []
for i in range(len(Y_test.columns)):
    accuracy = accuracy_score(Y_test.iloc[:, i], new_Y_pred[:, i])
    precision = precision_score(Y_test.iloc[:, i], new_Y_pred[:, i], average='micro')
    recall = recall_score(Y_test.iloc[:, i], new_Y_pred[:, i], average='micro')
    f1 = f1_score(Y_test.iloc[:, i], new_Y_pred[:, i], average='micro')
    new_metrics.append([accuracy, precision, recall, f1])

new_metrics_df = pd.DataFrame(data=np.array(new_metrics), index=list(Y_test.columns.values), columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
print(new_metrics_df)

                        Accuracy  Precision    Recall        F1
related                 0.791571   0.791571  0.791571  0.791571
request                 0.877765   0.877765  0.877765  0.877765
offer                   0.995423   0.995423  0.995423  0.995423
aid_related             0.734935   0.734935  0.734935  0.734935
medical_help            0.927346   0.927346  0.927346  0.927346
medical_products        0.954233   0.954233  0.954233  0.954233
search_and_rescue       0.970633   0.970633  0.970633  0.970633
security                0.981884   0.981884  0.981884  0.981884
military                0.966438   0.966438  0.966438  0.966438
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.948513   0.948513  0.948513  0.948513
food                    0.919908   0.919908  0.919908  0.919908
shelter                 0.926964   0.926964  0.926964  0.926964
clothing                0.985507   0.985507  0.985507  0.985507
money                   0.977689   0.977

In [11]:
metrics_df.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.942940,0.942940,0.942940,0.942940
std,0.060185,0.060185,0.060185,0.060185
min,0.736270,0.736270,0.736270,0.736270
25%,0.931445,0.931445,0.931445,0.931445
50%,0.958238,0.958238,0.958238,0.958238
75%,0.982838,0.982838,0.982838,0.982838
max,1.000000,1.000000,1.000000,1.000000


In [12]:
new_metrics_df.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.941023,0.941023,0.941023,0.941023
std,0.061127,0.061127,0.061127,0.061127
min,0.734935,0.734935,0.734935,0.734935
25%,0.927250,0.927250,0.927250,0.927250
50%,0.957475,0.957475,0.957475,0.957475
75%,0.982790,0.982790,0.982790,0.982790
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Reference: https://scikit-learn.org/stable/modules/multiclass.html

In [13]:
# Use multivariate Bernoulli models
from sklearn.naive_bayes import BernoulliNB

pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BernoulliNB()))
    ])

parameters2 = {
                'clf__estimator__fit_prior': [True, False],
                'clf__estimator__alpha': [0, 1, 2],
              }

cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
new_model2 = cv2.fit(X_train, Y_train)
new_model2.cv_results_

{'mean_fit_time': array([ 7.82808328,  7.80707971,  7.7793529 ,  7.74562248,  7.7300632 ,
         7.73425237]),
 'std_fit_time': array([ 0.04979496,  0.05515935,  0.09666187,  0.08116191,  0.14732038,
         0.12115649]),
 'mean_score_time': array([ 3.89354634,  3.87179923,  3.87382929,  3.84525561,  3.86776932,
         3.87469514]),
 'std_score_time': array([ 0.05081132,  0.0492018 ,  0.05359495,  0.05109898,  0.06861683,
         0.08158409]),
 'param_clf__estimator__alpha': masked_array(data = [0 0 1 1 2 2],
              mask = [False False False False False False],
        fill_value = ?),
 'param_clf__estimator__fit_prior': masked_array(data = [True False True False True False],
              mask = [False False False False False False],
        fill_value = ?),
 'params': [{'clf__estimator__alpha': 0, 'clf__estimator__fit_prior': True},
  {'clf__estimator__alpha': 0, 'clf__estimator__fit_prior': False},
  {'clf__estimator__alpha': 1, 'clf__estimator__fit_prior': True},
  {'c

In [14]:
new_model2.best_params_

{'clf__estimator__alpha': 1, 'clf__estimator__fit_prior': True}

In [15]:
new_Y_pred2 = new_model2.predict(X_test)

new_metrics2 = []
for i in range(len(Y_test.columns)):
    accuracy = accuracy_score(Y_test.iloc[:, i], new_Y_pred2[:, i])
    precision = precision_score(Y_test.iloc[:, i], new_Y_pred2[:, i], average='micro')
    recall = recall_score(Y_test.iloc[:, i], new_Y_pred2[:, i], average='micro')
    f1 = f1_score(Y_test.iloc[:, i], new_Y_pred2[:, i], average='micro')
    new_metrics2.append([accuracy, precision, recall, f1])

new_metrics_df2 = pd.DataFrame(data=np.array(new_metrics2), index=list(Y_test.columns.values), columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
print(new_metrics_df2)

                        Accuracy  Precision    Recall        F1
related                 0.810641   0.810641  0.810641  0.810641
request                 0.884630   0.884630  0.884630  0.884630
offer                   0.993326   0.993326  0.993326  0.993326
aid_related             0.742563   0.742563  0.742563  0.742563
medical_help            0.920862   0.920862  0.920862  0.920862
medical_products        0.950420   0.950420  0.950420  0.950420
search_and_rescue       0.968726   0.968726  0.968726  0.968726
security                0.979786   0.979786  0.979786  0.979786
military                0.963959   0.963959  0.963959  0.963959
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.936880   0.936880  0.936880  0.936880
food                    0.903890   0.903890  0.903890  0.903890
shelter                 0.917048   0.917048  0.917048  0.917048
clothing                0.983028   0.983028  0.983028  0.983028
money                   0.975210   0.975

In [16]:
new_metrics_df2.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.937177,0.937177,0.937177,0.937177
std,0.060516,0.060516,0.060516,0.060516
min,0.742563,0.742563,0.742563,0.742563
25%,0.916571,0.916571,0.916571,0.916571
50%,0.954615,0.954615,0.954615,0.954615
75%,0.980597,0.980597,0.980597,0.980597
max,1.000000,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [19]:
pickle.dump(new_model, open('disaster_pipeline.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.